<a href="https://colab.research.google.com/github/hws2002/MachineLearning_PytorchNScikitLearn/blob/master/chapter12/chapter12_2_pytorch_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import numpy as np

In [7]:
print(torch.__version__)

2.3.1+cu121


In [8]:
# 텐서 t_a, t_b는 shape = (3,) 속성과 원본 데이터에서 유도된 dtype = int32 속성과 함께 출력됨.
np.set_printoptions(precision=3)
a = [1,2,3]
b = np.array([4,5,6], dtype = np.int32)
t_a = torch.tensor(a)
t_b = torch.from_numpy(b)
print(t_a)
print(t_b)

tensor([1, 2, 3])
tensor([4, 5, 6], dtype=torch.int32)


In [10]:
t_ones = torch.ones(2,3)
t_ones.shape
print(t_ones)

tensor([[1., 1., 1.],
        [1., 1., 1.]])


In [11]:
# 상수 값을 가진 텐서를 다음과 같이 만들 수 있음
rand_tensor = torch.rand(2,3)
print(rand_tensor)

tensor([[0.0246, 0.3107, 0.4165],
        [0.8041, 0.7134, 0.3489]])


# 12.2.3. 텐서의 데이터 타입과 크기 조작
모델이나 연산에 맞는 입력을 준비하려면 텐서를 조작하는 방법을 배워야 함.
이 절에서 텐서의 데이터 타입과 크기를 조작하는 방법을 배워보자.
파이토치의 `to`, `reshape`, `transpose`, `squeeze(차원 삭제)` 함수를 사용함

## `torch.to()`
텐서의 데이터 타입을 원하는 타입으로 바꿈
* 다른 데이터 타입은 https://pytorch.org/docs/stable/tensor_attributes.html 를 참고하자

In [12]:
t_a_new = t_a.to(torch.int64)
print(t_a_new.dtype)

torch.int64


In [14]:
# 텐서 전치하기
t = torch.rand(3,5)
t_tr = torch.transpose(t, 0, 1)
print(t.shape, '-->', t_tr.shape)

torch.Size([3, 5]) --> torch.Size([5, 3])


In [15]:
# 텐서 크기 바꾸기 (예를 들어 1D 벡터에서 2D 배열로)
t = torch.zeros(30)
t_reshape = t.reshape(5,6)
print(t_reshape.shape)

torch.Size([5, 6])


In [18]:
# 불필요한 차원 삭제하기 (크기가 1인 차원은 불필요함)
t = torch.zeros(1,2,1,4,1)
t_sqz = torch.squeeze(t,2)
print(t.shape, '-->', t_sqz.shape)

torch.Size([1, 2, 1, 4, 1]) --> torch.Size([1, 2, 4, 1])


# 12.2.4 텐서에 수학 연산 적용
널리 사용하는 선형대수 연산을 배워보자.
원소별 곱셈, 행렬 곱셈, 텐서의 노름 연산 등등.

In [19]:
torch.manual_seed(1)
t1 = 2 * torch.rand(5,2) - 1 # [-1,1)사이의 균등 분포
t2 = torch.normal(mean = 0, std = 1, size = (5,2))

In [22]:
t3 = torch.multiply(t1,t2)
print(t3)

tensor([[ 0.4426, -0.3114],
        [ 0.0660, -0.5970],
        [ 1.1249,  0.0150],
        [ 0.1569,  0.7107],
        [-0.0451, -0.0352]])


In [28]:
# 특정 축(들)을 따라 평균, 합, 표준 편차를 계산할면 torch.mean(), torch.sum(), torch.std()를 사용할 수 있음
# 예를 들어 t1의 각 열 평균은 다음과 같이 계산 가능
t4 = torch.mean(t1, dim = 0) # or axis = 0
print(t4)

tensor([-0.1373,  0.2028])


In [30]:
# t1과 t2의 행렬 곱셈 은 torch.matmul() 함수를 사용하여 계산 가능
t5 = torch.matmul(t1,torch.transpose(t2,0,1))
print(t5)

tensor([[ 0.1312,  0.3860, -0.6267, -1.0096, -0.2943],
        [ 0.1647, -0.5310,  0.2434,  0.8035,  0.1980],
        [-0.3855, -0.4422,  1.1399,  1.5558,  0.4781],
        [ 0.1822, -0.5771,  0.2585,  0.8676,  0.2132],
        [ 0.0330,  0.1084, -0.1692, -0.2771, -0.0804]])


In [32]:
# torch.linalg.norm() 함수를 사용하여 텐서의 L^p norm을 게산할 수 있음
# 예를 들어 t1의 노름 L2는
norm_t1 = torch.linalg.norm(t1,ord = 2, dim = 1)
print(norm_t1)

tensor([0.6785, 0.5078, 1.1162, 0.5488, 0.1853])


In [35]:
# 이 값과 비교해보자
print(np.sqrt(np.sum(t1.numpy() ** 2, axis = 1)))

[0.678 0.508 1.116 0.549 0.185]


# 12.2.5 chunk(), stack(), cat() 함수
하나의 텐서를 여러 개의 텐서로 나누는 텐서플로 연산을 알아보자  
또는 반대로 여러 개의 텐서를 쌓거나 연결하여 하나의 텐서로 만드는 연산도 알아보자


In [41]:
# 하나의 텐서를 두 개 이상의 텐서로 나누어야 한다면, torch.chunk() 를 쓰자
# 입력된 텐서를 동일한 크기의 텐서 리스트로 나눔
# 두 번째 매개변수인 chunks 매개변수에 분할할 텐서 개수를 지정하고
# dim 매개변수로 원하는 차원을 지정할 수 있음
# 또는 torch.split()를 사용하여 원하는 크기를 리스트로 전달 할 수도 있음.

# 분할 개수 지정하기
torch.manual_seed(1)
t = torch.rand(6)
print(t)

t_splits = torch.chunk(t,3)
[item.numpy() for item in t_splits]
# !텐서 크기를 chunks 값으로 나눌 수 없는 경우 마지막에 작은 크기의 청크가 만들어짐

tensor([0.7576, 0.2793, 0.4031, 0.7347, 0.0293, 0.7999])


[array([0.758, 0.279], dtype=float32),
 array([0.403, 0.735], dtype=float32),
 array([0.029, 0.8  ], dtype=float32)]

In [43]:
# 다른 분할 크기 전달하기
# 분할 개수 대신에 출력 텐서의 크기를 직접 지정할 수도 있음
# 크기 5인 텐서를 크기 3과 2인 텐서 두개로 나눠보자.
torch.manual_seed(1)
t = torch.rand(5)
print(t)

t_splits = torch.split(t, split_size_or_sections=[3,2])
[item.numpy() for item in t_splits]

tensor([0.7576, 0.2793, 0.4031, 0.7347, 0.0293])


[array([0.758, 0.279, 0.403], dtype=float32),
 array([0.735, 0.029], dtype=float32)]

In [53]:
# 이따금 여러 개의 텐서를 연결하거나 쌓아서 하나의 텐서를 만들어야 하는 경우가 있음
# 이런 경우 torch.stack() 과 torch.cat() 같은 파이토치 함수를 사용하면 편리함

# 예를 들어 크기가 3이고 1로 채워진 1D텐서 A와 크기가 2이고 0으로 채워진 1D텐서 B가 있다고 가정해보자.
# 두 텐서를 가로로 연결하여 크기가 5인 1D텐서 C를 만들어보자
A = torch.ones(3)
B = torch.zeros(2)
C = torch.cat( (A,B), axis = 0)
print(C)

tensor([1., 1., 1., 0., 0.])


In [55]:
# 두 텐서를 세로로 연결해보자
B = torch.zeros(3)
C = torch.stack((A,B), axis = 1)
print(C)

tensor([[1., 0.],
        [1., 0.],
        [1., 0.]])


이 외 핵심적인 함수들은 파이토치 문서를 참고하자
https://pytorch.org/docs/stable/index.html